In [ ]:
import os
import os.path
import sys
import glob
from sys import platform
from shutil import copyfile
import json
import errno, stat, shutil
from shutil import copyfile
import numpy as np
from scipy.io import savemat
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
from scipy.ndimage import gaussian_filter
from tifffile import imread, TiffFile, imsave
import cv2
import warnings
import timeit
from roipoly import RoiPoly
import pickle
from datetime import datetime, date
from sys import platform
import glob
import traceback
from shutil import copy2
import re
import numpy as np
import copy
import shutil
import pickle
import time
import timeit
import subprocess
import gc
import tracemalloc
from termcolor import colored, cprint
import importlib
from skimage.registration import phase_cross_correlation
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.rcParams.update({'font.size': 20, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'pdf.fonttype':42})
from ScanImageTiffReader import ScanImageTiffReader

#####################################################################
#####################################################################
tif_rearrange_param={}
#####################################################################
#####################################################################
print("Finished package prep")

In [ ]:
##################################################################################################
def load_tiff_stack(path):
    return ScanImageTiffReader(path).data()
##################################################################################################
def write_tif(filename, data, RGB=False):
    d = len(data.shape);
    if d == 2:
        imsave(filename, data);
    elif d == 3:   
        imsave(filename, data.transpose([2,0,1]));
    elif d == 4:        
        dim0,dim1,dim2,dim3=data.shape
        if RGB:
            if dim3==3:
                imsave(filename, data.transpose([2,0,1,3]));
            elif dim2==3:
                imsave(filename, data.transpose([3,0,1,2]));
            else:
                imsave(filename, data);
        else:
            imsave(filename, data.transpose([2,0,1,3]));

    else:
        raise RuntimeError('writing multiple channel data to tif not supported!');
##################################################################################################
def imshow_cleanup(ax):
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_facecolor('none')
    return ax
##################################################################################################
#Main EOD rearrangement function
def rearrange_EOD_image (input_image,tif_rearrange_param,verbose):
    #####################################################################
    # Remove Noise
    input_image[input_image<=tif_rearrange_param["noise_floor"]]=np.uint(0)
    if verbose:
        fig1=plt.figure(1,figsize=(30,10))
        plt.imshow(input_image)
        plt.title("Input")
        plt.show(fig1)
    #####################################################################
    #Invert every other line
    invert_line_image=np.zeros((tif_rearrange_param["import_height"],tif_rearrange_param["import_width"]))
    if tif_rearrange_param["flip_alt_rows"]==2:
        for y in range(tif_rearrange_param["import_height"]):
            if y % 2 == 0:
                invert_line_image[y,:] = np.flip(input_image[y,:], axis = 0)
            else:
                invert_line_image[y,:] = input_image[y,:]
    elif tif_rearrange_param["flip_alt_rows"]==1:
        for y in range(tif_rearrange_param["import_height"]):
            if y % 2 == 0:
                invert_line_image[y,:] = input_image[y,:]
            else:
                invert_line_image[y,:] = np.flip(input_image[y,:], axis = 0)
    else:
        invert_line_image = input_image
    if verbose:
        fig2=plt.figure(1,figsize=(30,10))
        plt.imshow(invert_line_image)
        plt.title("Invert Line")
        plt.show(fig2)
    #####################################################################
    #phase shift lines
    phase_shift_image=np.zeros((tif_rearrange_param["import_height"],tif_rearrange_param["import_width"]))
    if tif_rearrange_param["line_shift"] != 0:
        if tif_rearrange_param["line_shift_alt"] == 2:
            #######################
            #only shifting half of the lines
            for y in range(tif_rearrange_param["import_height"]):
                if y % 2 == 0:
                    phase_shift_image[y,range(0,tif_rearrange_param["import_width"]-tif_rearrange_param["line_shift"])]=invert_line_image[y,range(tif_rearrange_param["line_shift"],tif_rearrange_param["import_width"])]
                    if y>0:
                        phase_shift_image[y-1,range(tif_rearrange_param["import_width"]-tif_rearrange_param["line_shift"],tif_rearrange_param["import_width"])]= \
                        np.flip(invert_line_image[y-1,range(0,tif_rearrange_param["line_shift"])], axis = 0)
                else:
                    phase_shift_image[y,:]=invert_line_image[y,:]
        elif tif_rearrange_param["line_shift_alt"] == 1:
            #######################
            #only shifting half of the lines
            for y in range(tif_rearrange_param["import_height"]):
                if y % 2 == 0:
                    phase_shift_image[y,:]=invert_line_image[y,:]
                else:
                    phase_shift_image[y,range(0,tif_rearrange_param["import_width"]-tif_rearrange_param["line_shift"])]=invert_line_image[y,range(tif_rearrange_param["line_shift"],tif_rearrange_param["import_width"])]
                    if y>0:
                        phase_shift_image[y-1,range(tif_rearrange_param["import_width"]-tif_rearrange_param["line_shift"],tif_rearrange_param["import_width"])]= \
                        np.flip(invert_line_image[y-1,range(0,tif_rearrange_param["line_shift"])], axis = 0)
        else:
            if tif_rearrange_param["line_shift"] != 0:
                #######################
                #start by extracting a continuous list of pixels
                num_pixels=int(tif_rearrange_param["import_height"]*tif_rearrange_param["import_width"])
                for y in range(tif_rearrange_param["import_height"]):
                    if y==0:
                        pixellist=invert_line_image[y,:]
                    else:
                        pixellist=np.concatenate((pixellist,invert_line_image[y,:]))
                #######################
                #Then shift pixels and fill with zeros
                if tif_rearrange_param["line_shift"]>0:
                    pixellist[range(tif_rearrange_param["line_shift"],num_pixels)]=pixellist[range(0,num_pixels-tif_rearrange_param["line_shift"])]
                    pixellist[range(0,tif_rearrange_param["line_shift"])]=0
                elif tif_rearrange_param["line_shift"]<0:
                    pixellist[range(0,num_pixels-abs(tif_rearrange_param["line_shift"]))]=pixellist[range(abs(tif_rearrange_param["line_shift"]),num_pixels)]
                    pixellist[range(num_pixels-abs(tif_rearrange_param["line_shift"]),num_pixels)]=0
                #######################
                #Then reconstruct image
                phase_shift_image=np.zeros((tif_rearrange_param["import_height"],tif_rearrange_param["import_width"]))
                for y in range(tif_rearrange_param["import_height"]):
                    phase_shift_image[y,:]=pixellist[range(y*tif_rearrange_param["import_width"],(y+1)*tif_rearrange_param["import_width"])]
    else:
        phase_shift_image=invert_line_image
    if tif_rearrange_param["truncate_column"]:
        phase_shift_image=phase_shift_image[:,range(tif_rearrange_param["import_width"]-1)]            
    #####################################################################
    intermediate_image_height,intermediate_image_width=phase_shift_image.shape
    if verbose:
        fig2=plt.figure(1,figsize=(30,10))
        plt.imshow(phase_shift_image)
        plt.title("PhaseShift")
        plt.show(fig2)
        print("Intermediate Image Height = ",intermediate_image_height)
        print("Intermediate Image Width = ",intermediate_image_width)
    #####################################################################
    #Truncate single pixel
    # simple cut in half
    # second half gets flipped and interleaveed 
    if tif_rearrange_param["simple_interleave"]:
        split_interleave_image=np.zeros((int(np.floor(intermediate_image_height*2)),int(np.floor(intermediate_image_width/2))))
        split_interleave_image_height,split_interleave_image_width=split_interleave_image.shape
        linecount=0
        for y in range(intermediate_image_height):
            split_interleave_image[linecount,:] = phase_shift_image[y,range(0,split_interleave_image_width)]
            linecount+=1
            if tif_rearrange_param["flip_simple_interleave"]:
                split_interleave_image[linecount,:] = \
                np.flip(phase_shift_image[y,range(split_interleave_image_width,split_interleave_image_width*2)], axis = 0)
            else:
                split_interleave_image[linecount,:] = phase_shift_image[y,range(split_interleave_image_width,split_interleave_image_width*2)]
            linecount+=1
    else:
        split_interleave_image=phase_shift_image
        split_interleave_image_height,split_interleave_image_width=split_interleave_image.shape
    #####################################################################
    #customized interleave
    if tif_rearrange_param["offset_split_interleave"]>0:
        if tif_rearrange_param["offset_split_interleave"]==3:
            max_width=max([max(tif_rearrange_param["offset_split_interleave_param"]["First Placement"]),max(tif_rearrange_param["offset_split_interleave_param"]["Second Placement"]),max(tif_rearrange_param["offset_split_interleave_param"]["Third Placement"]),max(tif_rearrange_param["offset_split_interleave_param"]["Fourth Placement"])])+1
            max_height=int(np.floor(intermediate_image_height*4))
        elif tif_rearrange_param["offset_split_interleave"]==2:
            max_width=max([max(tif_rearrange_param["offset_split_interleave_param"]["First Placement"]),max(tif_rearrange_param["offset_split_interleave_param"]["Second Placement"]),max(tif_rearrange_param["offset_split_interleave_param"]["Third Placement"])])+1
            max_height=int(np.floor(intermediate_image_height*3))
        elif tif_rearrange_param["offset_split_interleave"]==1:
            max_height=int(np.floor(intermediate_image_height*2))
            max_width=max([max(tif_rearrange_param["offset_split_interleave_param"]["First Placement"]),max(tif_rearrange_param["offset_split_interleave_param"]["Second Placement"])])+1
        
        tif_rearrange_param["offset_split_interleave_image"] = np.zeros((max_height,max_width))
        if tif_rearrange_param["offset_split_collapse"]:
            tif_rearrange_param["offset_split_interleave_image"][tif_rearrange_param["offset_split_interleave_image"] == 0] = 'nan'
        linecount=0
        for y in range(intermediate_image_height):
            if tif_rearrange_param["offset_split_interleave_param"]["First Invert"]:
                tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["First Placement"]] = \
                np.flip(split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["First Source"]], axis = 0)
            else:
                tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["First Placement"]] = \
                split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["First Source"]]
            linecount+=1
            if tif_rearrange_param["offset_split_interleave"]>=1:
                if tif_rearrange_param["offset_split_interleave_param"]["Second Invert"]:
                    tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["Second Placement"]] = \
                    np.flip(split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["Second Source"]], axis = 0)
                else:
                    tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["Second Placement"]] = \
                    split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["Second Source"]]
                linecount+=1
            if tif_rearrange_param["offset_split_interleave"]>=2:
                if tif_rearrange_param["offset_split_interleave_param"]["Third Invert"]:
                    tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["Third Placement"]] = \
                    np.flip(split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["Third Source"]], axis = 0)
                else:
                    tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["Third Placement"]] = \
                    split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["Third Source"]]
                linecount+=1
            if tif_rearrange_param["offset_split_interleave"]>=3:
                if tif_rearrange_param["offset_split_interleave_param"]["Fourth Invert"]:
                    tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["Fourth Placement"]] = \
                    np.flip(split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["Fourth Source"]], axis = 0)
                else:
                    tif_rearrange_param["offset_split_interleave_image"][linecount,tif_rearrange_param["offset_split_interleave_param"]["Fourth Placement"]] = \
                    split_interleave_image[y,tif_rearrange_param["offset_split_interleave_param"]["Fourth Source"]]
                linecount+=1
        if tif_rearrange_param["offset_split_collapse"]:
            tif_rearrange_param["offset_split_interleave_image_height"],tif_rearrange_param["offset_split_interleave_image_width"]=tif_rearrange_param["offset_split_interleave_image"].shape
            collapse_rows=tif_rearrange_param["offset_split_interleave"]+1
            collapse_line_count=0
            collapsed_position=0
            post_collapse_height=int(tif_rearrange_param["offset_split_interleave_image_height"]/(tif_rearrange_param["offset_split_interleave"]+1))
            offset_split_interleave_image_collapsed = np.zeros((post_collapse_height,max_width))
            for y1 in range(post_collapse_height):
                temp_range=range(collapse_line_count,collapse_line_count+collapse_rows)
                collapse_line_count=collapse_line_count+collapse_rows
                offset_split_interleave_image_collapsed[collapsed_position,:]=\
                np.nanmean(tif_rearrange_param["offset_split_interleave_image"][temp_range,:],axis = 0)
                collapsed_position+=1            
            tif_rearrange_param["offset_split_interleave_image"]=offset_split_interleave_image_collapsed
    else:
        tif_rearrange_param["offset_split_interleave_image"] = split_interleave_image
    #####################################################################
    if tif_rearrange_param["flip_all_rows"] and tif_rearrange_param["offset_split_interleave"]==1:
        tif_rearrange_param["offset_split_interleave_image2"] = copy.deepcopy(tif_rearrange_param["offset_split_interleave_image"])
        for y in range(int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1)))):
            tif_rearrange_param["offset_split_interleave_image2"][(tif_rearrange_param["offset_split_interleave"]+1)*(y),:] = tif_rearrange_param["offset_split_interleave_image"][(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1,:]
            tif_rearrange_param["offset_split_interleave_image2"][(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1,:] = tif_rearrange_param["offset_split_interleave_image"][(tif_rearrange_param["offset_split_interleave"]+1)*(y),:]
    elif tif_rearrange_param["flip_all_rows"] and not tif_rearrange_param["offset_split_interleave"]==1:
        print("ERROR TURN OFF tif_rearrange_param['flip_all_rows']")
    else:
        tif_rearrange_param["offset_split_interleave_image2"] = copy.deepcopy(tif_rearrange_param["offset_split_interleave_image"])
    #####################################################################
    final_image=tif_rearrange_param["offset_split_interleave_image2"]
    tif_rearrange_param["final_image_height"],tif_rearrange_param["final_image_width"]=final_image.shape
    if verbose:
        fig3=plt.figure(1,figsize=(30,10))
        plt.imshow(tif_rearrange_param["offset_split_interleave_image"])
        plt.title("Final Image")
        plt.show(fig3)
        print("Final Image Height = ",tif_rearrange_param["final_image_height"])
        print("Final Image Width = ",tif_rearrange_param["final_image_width"])
    #####################################################################   
    final_image=np.array(final_image)
    return final_image, tif_rearrange_param["final_image_height"], tif_rearrange_param["final_image_width"]
##########################################################################################################################################
def single_tif_rearrange(fileID,testfilecount,input_image_array,tif_file_list,tif_rearrange_param,verbose=True):
    if not 'frameRange' in tif_rearrange_param:
        tif_rearrange_param["frameRange"] = [0,input_image_array.shape[2]]
    tif_rearrange_param["frames"] = list(range(tif_rearrange_param["frameRange"][0],tif_rearrange_param["frameRange"][1]))
    tif_rearrange_param["nframes"] = len(tif_rearrange_param["frames"])
    #################################################################
    input_image_array_resize = np.zeros((int(np.round(input_image_array.shape[0]*tif_rearrange_param["initial_resize_y"])),
                                            int(np.round(input_image_array.shape[1]*tif_rearrange_param["initial_resize_x"])),
                                            tif_rearrange_param["nframes"]),dtype='float32')
    if tif_rearrange_param["initial_resize"]:
        if verbose:
            print('Initial resizing data in y by '+str(tif_rearrange_param["initial_resize_y"])+" and x by "+str(tif_rearrange_param["initial_resize_x"]))
        for f,frame in enumerate(tif_rearrange_param["frames"]):
            input_image_array_resize[:,:,f] = cv2.resize(input_image_array[:,:,frame], dsize=(int(np.round(input_image_array.shape[1]*tif_rearrange_param["initial_resize_x"])),\
                                                                                        int(np.round(input_image_array.shape[0]*tif_rearrange_param["initial_resize_y"]))), \
                                                                        interpolation=tif_rearrange_param["initial_resize_method"])
    else:
        for f,frame in enumerate(tif_rearrange_param["frames"]):
            input_image_array_resize[:,:,f] = copy.deepcopy(input_image_array[:,:,frame])
    #################################################################
    if "fixed_image_array" in locals():
        del fixed_image_array
    temp_image, fixed_image_height, fixed_image_width = \
        rearrange_EOD_image(input_image_array_resize[:,:,0],tif_rearrange_param,False)
    fixed_image_array=np.zeros((fixed_image_height, fixed_image_width,tif_rearrange_param["nframes"]))
    for f,frame in enumerate(tif_rearrange_param["frames"]):
        temp_image, fixed_image_height, fixed_image_width = \
            rearrange_EOD_image(input_image_array_resize[:,:,f],tif_rearrange_param,False)
        fixed_image_array[:,:,f]=copy.deepcopy(temp_image)
    #################################################################
    del input_image_array_resize
    fixed_overall_mean_image=np.mean(fixed_image_array,axis=2)
    if tif_rearrange_param["intermediate_resize"]:
        if verbose:
            print('Intermediate resizing data in y by '+str(tif_rearrange_param["intermediate_resize_y"])+" and x by "+str(tif_rearrange_param["intermediate_resize_x"]))
        fixed_overall_mean_image = cv2.resize(fixed_overall_mean_image, dsize=(int(np.round(fixed_overall_mean_image.shape[1]*tif_rearrange_param["intermediate_resize_x"])),\
                                                                                    int(np.round(fixed_overall_mean_image.shape[0]*tif_rearrange_param["intermediate_resize_y"]))), \
                                                                    interpolation=tif_rearrange_param["intermediate_resize_method"])
    #################################################################
    if tif_rearrange_param["offset_split_interleave"] == 1:
        if tif_rearrange_param["manual_bidir_linepair_shift_and_scale"]:
            for y in range(int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1)))):
                idx1=(tif_rearrange_param["offset_split_interleave"]+1)*(y)
                idx2=(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1
                line1=copy.deepcopy(fixed_overall_mean_image[idx1,:])
                line2=copy.deepcopy(fixed_overall_mean_image[idx2,:])
                if 'list' in str(type(tif_rearrange_param["manual_bidir_linepair_shift"])):
                    shiftVal=tif_rearrange_param["manual_bidir_linepair_shift"][y]
                else:
                    shiftVal=tif_rearrange_param["manual_bidir_linepair_shift"]
                if 'list' in str(type(tif_rearrange_param["manual_bidir_linepair_shift"])):
                    resize_factor=tif_rearrange_param["manual_bidir_linepair_resize_factor"][y]
                else:
                    resize_factor=tif_rearrange_param["manual_bidir_linepair_resize_factor"]
                line2_shift=np.roll(line2,shiftVal)
                if shiftVal>0:
                    line2_shift[0:shiftVal] = 0
                elif shiftVal<0:
                    line2_shift[len(line2_shift)-shiftVal-1:len(line2_shift)] = 0
                line2_resize = int(np.round(line2_shift.shape[0]*resize_factor))
                line2_shift = np.squeeze(cv2.resize(np.expand_dims(line2_shift,1), dsize=(1,line2_resize), \
                                                    interpolation=tif_rearrange_param["manual_bidir_linepair_resize_method"]))
                if line2_shift.shape[0] >= fixed_overall_mean_image.shape[1]:
                    fixed_overall_mean_image[idx2,:] = copy.deepcopy(line2_shift[0:fixed_overall_mean_image.shape[1]])
                else:
                    fixed_overall_mean_image[idx2,0:line2_shift.shape[0]] = copy.deepcopy(line2_shift)

                if 'list' in str(type(tif_rearrange_param["manual_bidir_linepair_shift"])):
                    pairShift=tif_rearrange_param["manual_bidir_linepair_pairshifts"][y]
                else:
                    pairShift=tif_rearrange_param["manual_bidir_linepair_pairshifts"]

                line1=copy.deepcopy(fixed_overall_mean_image[idx1,:])
                line2=copy.deepcopy(fixed_overall_mean_image[idx2,:])
                line1_shift=np.roll(line1,pairShift)
                if pairShift>0:
                    line1_shift[0:pairShift] = 0
                elif pairShift<0:
                    line1_shift[len(line1_shift)-pairShift-1:len(line1_shift)] = 0
                if line1_shift.shape[0] >= fixed_overall_mean_image.shape[1]:
                    fixed_overall_mean_image[idx1,:] = copy.deepcopy(line1_shift[0:fixed_overall_mean_image.shape[1]])
                else:
                    fixed_overall_mean_image[idx1,0:line1_shift.shape[0]] = copy.deepcopy(line1_shift)
                line2_shift=np.roll(line2,pairShift)
                if pairShift>0:
                    line2_shift[0:pairShift] = 0
                elif pairShift<0:
                    line2_shift[len(line2_shift)-pairShift-1:len(line2_shift)] = 0
                if line2_shift.shape[0] >= fixed_overall_mean_image.shape[1]:
                    fixed_overall_mean_image[idx2,:] = copy.deepcopy(line2_shift[0:fixed_overall_mean_image.shape[1]])
                else:
                    fixed_overall_mean_image[idx2,0:line2_shift.shape[0]] = copy.deepcopy(line2_shift)
                if verbose:
                    print("Manual Shifting PAIR "+str(y)+" line "+str(idx2)+" shiftVal = "+str(shiftVal)+" resize_factor = "+str(resize_factor)+" pairShift = "+str(pairShift))
        #################################################################
        if tif_rearrange_param["line_pair_correlation_auto_shifting"]:
            for y in range(int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1)))):
                idx1=(tif_rearrange_param["offset_split_interleave"]+1)*(y)
                idx2=(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1
                line1=copy.deepcopy(fixed_overall_mean_image[idx1,:])
                line2=copy.deepcopy(fixed_overall_mean_image[idx2,:])
                if tif_rearrange_param["line_pair_correlation_auto_shifting_filter"] > 0:
                    line1_filt=np.convolve(line1, np.ones(int(tif_rearrange_param["line_pair_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["line_pair_correlation_auto_shifting_filter"])), 'same')
                    line2_filt=np.convolve(line2, np.ones(int(tif_rearrange_param["line_pair_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["line_pair_correlation_auto_shifting_filter"])), 'same')
                else:
                    line1_filt=copy.deepcopy(line1)
                    line2_filt=copy.deepcopy(line2)

                shift, error, diffphase = phase_cross_correlation(np.expand_dims(line1_filt,axis=1), \
                                                                np.expand_dims(line2_filt,axis=1), \
                                                                upsample_factor=1)
                tform = EuclideanTransform(translation=shift[::-1])
                shiftVal=shift[0]
                if np.absolute(shiftVal)<=tif_rearrange_param["line_pair_correlation_auto_shifting_max_shift"] and not np.absolute(shiftVal) == 0:
                    if verbose:
                        print("Auto Correlation Shifting PAIR "+str(y)+" line "+str(idx2)+" by "+str(shiftVal)+" pixels")
                    line2_shift=np.squeeze(warp(np.expand_dims(line2,axis=1), inverse_map=tform.inverse))
                else:
                    line2_shift = copy.deepcopy(line2)
                fixed_overall_mean_image[idx2,:] = copy.deepcopy(line2_shift)

    else:
        if tif_rearrange_param["line_pair_correlation_auto_shifting"]:
            print("WARNING UNABLE TO PERFORM THIS line_pair_correlation_auto_shifting with current offset_split_interleave_image Settings")
            print("WARNING UNABLE TO PERFORM THIS line_pair_correlation_auto_shifting with current offset_split_interleave_image Settings")
            print("WARNING UNABLE TO PERFORM THIS line_pair_correlation_auto_shifting with current offset_split_interleave_image Settings")
            print("WARNING UNABLE TO PERFORM THIS line_pair_correlation_auto_shifting with current offset_split_interleave_image Settings")
            print("WARNING UNABLE TO PERFORM THIS line_pair_correlation_auto_shifting with current offset_split_interleave_image Settings")
    if tif_rearrange_param["offset_split_interleave"] == 1:
        if tif_rearrange_param["inter_line_pair_correlation_auto_shifting"]:
            tif_rearrange_param["inter_line_pair_correlation_auto_shifting_max_shift"] = 50
            if tif_rearrange_param["inter_line_pair_correlation_auto_shifting_direction"] == 0:
                for y in range(0,int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1)))-1):
                    idx1a=(tif_rearrange_param["offset_split_interleave"]+1)*(y)
                    idx1b=(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1
                    idx2a=(tif_rearrange_param["offset_split_interleave"]+1)*(y+1)
                    idx2b=(tif_rearrange_param["offset_split_interleave"]+1)*(y+1)+1

                    line1=np.nanmean(fixed_overall_mean_image[idx1a:idx1b,:],axis=0)
                    line2=np.nanmean(fixed_overall_mean_image[idx2a:idx2b,:],axis=0)
                    line1a=copy.deepcopy(fixed_overall_mean_image[idx1a,:])
                    line1b=copy.deepcopy(fixed_overall_mean_image[idx1b,:])
                    line2a=copy.deepcopy(fixed_overall_mean_image[idx2a,:])
                    line2b=copy.deepcopy(fixed_overall_mean_image[idx2b,:])
                    if tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"] > 0:
                        line1_filt=np.convolve(line1, np.ones(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])), 'same')
                        line2_filt=np.convolve(line2, np.ones(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])), 'same')
                    else:
                        line1_filt=copy.deepcopy(line1)
                        line2_filt=copy.deepcopy(line2)
                    shift, error, diffphase = phase_cross_correlation(np.expand_dims(line1_filt,axis=1), \
                                                                    np.expand_dims(line2_filt,axis=1), \
                                                                    upsample_factor=1)
                    tform = EuclideanTransform(translation=shift[::-1])
                    shiftVal=shift[0]
                    if np.absolute(shiftVal)<=tif_rearrange_param["inter_line_pair_correlation_auto_shifting_max_shift"] and not np.absolute(shiftVal) == 0:
                        if verbose:
                            print("Auto Correlation Shifting PAIR "+str(y)+" lines "+str(idx2a)+" and "+str(idx2b)+" by "+str(shiftVal)+" pixels")
                        line2a_shift=np.squeeze(warp(np.expand_dims(line2a,axis=1), inverse_map=tform.inverse))
                        line2b_shift=np.squeeze(warp(np.expand_dims(line2b,axis=1), inverse_map=tform.inverse))
                    else:
                        line2a_shift = copy.deepcopy(line2a)
                        line2b_shift = copy.deepcopy(line2b)
                    fixed_overall_mean_image[idx2a,:] = copy.deepcopy(line2a_shift)
                    fixed_overall_mean_image[idx2b,:] = copy.deepcopy(line2b_shift)
            else:
                for y in reversed(range(1,int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1))))):
                    idx1a=(tif_rearrange_param["offset_split_interleave"]+1)*(y)
                    idx1b=(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1
                    idx2a=(tif_rearrange_param["offset_split_interleave"]+1)*(y-1)
                    idx2b=(tif_rearrange_param["offset_split_interleave"]+1)*(y-1)+1
                    
                    line1=np.nanmean(fixed_overall_mean_image[idx1a:idx1b,:],axis=0)
                    line2=np.nanmean(fixed_overall_mean_image[idx2a:idx2b,:],axis=0)
                    line1a=copy.deepcopy(fixed_overall_mean_image[idx1a,:])
                    line1b=copy.deepcopy(fixed_overall_mean_image[idx1b,:])
                    line2a=copy.deepcopy(fixed_overall_mean_image[idx2a,:])
                    line2b=copy.deepcopy(fixed_overall_mean_image[idx2b,:])
                    if tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"] > 0:
                        line1_filt=np.convolve(line1, np.ones(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])), 'same')
                        line2_filt=np.convolve(line2, np.ones(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"])), 'same')
                    else:
                        line1_filt=copy.deepcopy(line1)
                        line2_filt=copy.deepcopy(line2)
                    
                    shift, error, diffphase = phase_cross_correlation(np.expand_dims(line1_filt,axis=1), \
                                                                    np.expand_dims(line2_filt,axis=1), \
                                                                    upsample_factor=1)
                    tform = EuclideanTransform(translation=shift[::-1])
                    shiftVal=shift[0]
                    if np.absolute(shiftVal)<=tif_rearrange_param["inter_line_pair_correlation_auto_shifting_max_shift"] and not np.absolute(shiftVal) == 0:
                        if verbose:
                            print("Auto Correlation Shifting PAIR "+str(y)+" lines "+str(idx2a)+" and "+str(idx2b)+" by "+str(shiftVal)+" pixels")
                        line2a_shift=np.squeeze(warp(np.expand_dims(line2a,axis=1), inverse_map=tform.inverse))
                        line2b_shift=np.squeeze(warp(np.expand_dims(line2b,axis=1), inverse_map=tform.inverse))
                    else:
                        line2a_shift = copy.deepcopy(line2a)
                        line2b_shift = copy.deepcopy(line2b)
                    fixed_overall_mean_image[idx2a,:] = copy.deepcopy(line2a_shift)
                    fixed_overall_mean_image[idx2b,:] = copy.deepcopy(line2b_shift)
    if tif_rearrange_param["all_line_correlation_auto_shifting"]:
        if tif_rearrange_param["all_line_correlation_auto_shifting_direction"] == 0:
            for y in range(0,fixed_overall_mean_image.shape[0]-1):
                idx1=y
                idx2=y+1

                line1=copy.deepcopy(fixed_overall_mean_image[idx1,:])
                line2=copy.deepcopy(fixed_overall_mean_image[idx2,:])

                if tif_rearrange_param["all_line_correlation_auto_shifting_filter"] > 0:
                    line1_filt=np.convolve(line1, np.ones(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])), 'same')
                    line2_filt=np.convolve(line2, np.ones(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])), 'same')
                else:
                    line1_filt=copy.deepcopy(line1)
                    line2_filt=copy.deepcopy(line2)

                shift, error, diffphase = phase_cross_correlation(np.expand_dims(line1_filt,axis=1), \
                                                                np.expand_dims(line2_filt,axis=1), \
                                                                upsample_factor=1)
                tform = EuclideanTransform(translation=shift[::-1])
                shiftVal=shift[0]
                if np.absolute(shiftVal)<=tif_rearrange_param["all_line_correlation_auto_shifting_max_shift"] and not np.absolute(shiftVal) == 0:
                    if verbose:
                        print("ALL LINE AUTO Shifting lines "+str(idx1)+" and "+str(idx2)+" by "+str(shiftVal)+" pixels")
                    line2_shift=np.squeeze(warp(np.expand_dims(line2,axis=1), inverse_map=tform.inverse))
                else:
                    line2_shift = copy.deepcopy(line2)

                fixed_overall_mean_image[idx2,:] = copy.deepcopy(line2_shift)

        else:
            for y in reversed(range(1,fixed_overall_mean_image.shape[0])):
                idx1=y
                idx2=y-1
                line1=copy.deepcopy(fixed_overall_mean_image[idx1,:])
                line2=copy.deepcopy(fixed_overall_mean_image[idx2,:])
                if tif_rearrange_param["all_line_correlation_auto_shifting_filter"] > 0:
                    line1_filt=np.convolve(line1, np.ones(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])), 'same')
                    line2_filt=np.convolve(line2, np.ones(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])) / float(int(tif_rearrange_param["all_line_correlation_auto_shifting_filter"])), 'same')
                else:
                    line1_filt=copy.deepcopy(line1)
                    line2_filt=copy.deepcopy(line2)

                    shift, error, diffphase = phase_cross_correlation(np.expand_dims(line1_filt,axis=1), \
                                                                np.expand_dims(line2_filt,axis=1), \
                                                                upsample_factor=1)
                tform = EuclideanTransform(translation=shift[::-1])
                shiftVal=shift[0]
                if np.absolute(shiftVal)<=tif_rearrange_param["all_line_correlation_auto_shifting_max_shift"] and not np.absolute(shiftVal) == 0:
                    if verbose:
                        print("ALL LINE AUTO Shifting lines "+str(idx1)+" and "+str(idx2)+" by "+str(shiftVal)+" pixels")
                    line2_shift=np.squeeze(warp(np.expand_dims(line2,axis=1), inverse_map=tform.inverse))
                else:
                    line2_shift = copy.deepcopy(line2)
                fixed_overall_mean_image[idx2,:] = copy.deepcopy(line2_shift)
    #################################################################
    if tif_rearrange_param["offset_split_interleave"] == 1:
        if tif_rearrange_param["manual_bidir_linepair_amp_adjust"]:
            if 'list' in str(type(tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"])):
                if verbose:
                    print("Line Pair Amplitude Adjustments = "+str(tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"]))
            for y in range(int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1)))):
                if 'numpy' in str(type(tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"])):
                    ampAdjust = copy.deepcopy(tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"][y,:])
                    if verbose:
                        print("Line Pair "+str(y)+" Amplitude Adjustments = "+str(ampAdjust))
                else:
                    ampAdjust=copy.deepcopy(tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"])
                idx1=(tif_rearrange_param["offset_split_interleave"]+1)*(y)
                idx2=(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1
                fixed_overall_mean_image[idx1,:] = fixed_overall_mean_image[idx1,:] * ampAdjust[0]
                fixed_overall_mean_image[idx2,:] = fixed_overall_mean_image[idx2,:] * ampAdjust[1]

    if verbose:
        for y in range(int(np.floor(tif_rearrange_param["offset_split_interleave_image"].shape[0]/(tif_rearrange_param["offset_split_interleave"]+1)))):
            idx1=(tif_rearrange_param["offset_split_interleave"]+1)*(y)
            idx2=(tif_rearrange_param["offset_split_interleave"]+1)*(y)+1
            line1=copy.deepcopy(fixed_overall_mean_image[idx1,:])
            line2=copy.deepcopy(fixed_overall_mean_image[idx2,:])
            plt.figure(figsize=(40,3))
            
            plt.plot(line1,label='line1',color=(0,0,0))
            plt.plot(line2,label='line2',color=(1,0,0))
            plt.legend(fontsize= 20)
            plt.title("Line Pair "+str(y),fontsize=20)
            plt.minorticks_on()
            plt.xlim([0,np.max([line1.shape[0],line2.shape[0]])])

            plt.grid(visible=True, which='both', axis='x')

    if tif_rearrange_param["final_resize"]:
        if verbose:
            print('FINAL resizing data in y by '+str(tif_rearrange_param["final_resize_y"])+" and x by "+str(tif_rearrange_param["final_resize_x"]))
        fixed_overall_mean_image = np.squeeze(cv2.resize(fixed_overall_mean_image, dsize=(int(np.round(fixed_overall_mean_image.shape[1]*tif_rearrange_param["final_resize_x"])),int(np.round(fixed_overall_mean_image.shape[0]*tif_rearrange_param["final_resize_y"]))), \
                                            interpolation=tif_rearrange_param["final_resize_method"]))

    return fixed_image_array, fixed_overall_mean_image, fixed_image_height, fixed_image_widt
##########################################################################################################################################

In [ ]:
#Pick your directory with all the data
tif_rearrange_param["tif_import_folder"] = 'K:\\Farinella_2024_repo\\raw_data\\Fig6\\'
#####################################################################
#####################################################################
list_of_files = sorted( filter( os.path.isfile,
                        glob.glob(tif_rearrange_param["tif_import_folder"] + '*.tif') ) )
#Find all .tif files
numfiles=len(list_of_files)
tif_file_list={}
filecount=0
for f in range(numfiles):
    filecount+=1
    tif_file_list[filecount]=list_of_files[f]
    if filecount<=20:
        print("File # ",filecount," = ",tif_file_list[filecount])
    elif filecount==101:
        print("Not displaying all entries...")
print("I found ",str(filecount)," files")
#####################################################################
#####################################################################

In [ ]:
resort_data=True
startStr='D280_beads_50prct_'
endStr='_00001'

positions=list()
pos=-1
for f,file in enumerate(tif_file_list):
    if resort_data:
        pos=tif_file_list[file][tif_file_list[file].find(startStr)+len(startStr):tif_file_list[file].find(endStr)]
        pos=pos.replace('p','.')
    else:
        pos+=1
    positions.append(float(pos))
sorting_idx = [tif_file_list for _,tif_file_list in sorted(zip(positions,tif_file_list))]
tif_file_list1={}
for s0,s in enumerate(sorting_idx):
    tif_file_list1[s0] = copy.deepcopy(tif_file_list[s])
tif_file_list = copy.deepcopy(tif_file_list1)
del tif_file_list1
for f,file in enumerate(tif_file_list):
    print(tif_file_list[file])

In [ ]:
#####################################################################
fileID="211020_EOD280_beads_50prct_2umSteps"
#####################################################################
print("fileID: ",fileID)
for f,file in enumerate(tif_file_list):
    print(tif_file_list[file])

saveDir = 'K:\\Farinella_2024_repo\\processed_data\\Fig6\\'

In [ ]:
#Pull raw image info from one file
filecount=1
print("Loading: "+tif_file_list[filecount],end="...")
tif = TiffFile(tif_file_list[filecount])
tif_rearrange_param["import_num_frames"] = len(tif.pages)  # number of pages in the file
temppage=tif.pages[0]
tif_rearrange_param["import_height"],tif_rearrange_param["import_width"]=temppage.shape
tif.close()
print("Finished!")
print("IMPORT Image Height = ",tif_rearrange_param["import_height"])
print("IMPORT Image Width = ",tif_rearrange_param["import_width"])
print("IMPORT Num Frames = ",tif_rearrange_param["import_num_frames"])

In [ ]:
#####################################################################
#This is my standard meta format for some other scripts
ImagingInfo={}
ImagingInfo['PixelHeight']=0.4100986;
ImagingInfo['PixelWidth']=2.6246719;
ImagingInfo['PixelHeight']=13/40;
ImagingInfo['PixelWidth']=13/(353/2);
ImagingInfo['PixelRatio'] = np.round(ImagingInfo['PixelHeight'] /ImagingInfo['PixelWidth'],decimals=1)
ImagingInfo['PixelUnit']='um';
ImagingInfo['nSlices'] = len(tif_file_list)
ImagingInfo['VoxelDepth']=2;
ImagingInfo['VoxelUnit']='um';
ImagingInfo['InterFrameTime']=0.00015;
ImagingInfo['FrameUnit']='s';
#####################################################################
ImagingInfo['PixelRatio'] = np.round(ImagingInfo['PixelHeight'] /ImagingInfo['PixelWidth'],decimals=1)
ImagingInfo['VoxelRatio'] = np.round(ImagingInfo['VoxelDepth'] /ImagingInfo['PixelWidth'],decimals=1)
print(ImagingInfo['PixelRatio'])
print(ImagingInfo['VoxelRatio'])

In [ ]:
#Load Test Stack
testfilecount = 5
print("Loading: ",tif_file_list[testfilecount],end='...')
try:
    input_image_array = load_tiff_stack(tif_file_list[testfilecount])
except:    
    print("ScanImage load not working ",end='...')
    input_image_array=imread(tif_file_list[testfilecount])
print("Finished!")
input_image_array=np.moveaxis(input_image_array,[0, 1, 2], [-1, -3, -2])
tif_rearrange_param["import_height"],tif_rearrange_param["import_width"],tif_rearrange_param["import_num_frames"]=input_image_array.shape
print("IMPORT Image Height = ",tif_rearrange_param["import_height"])
print("IMPORT Image Width = ",tif_rearrange_param["import_width"])
print("IMPORT Num Frames = ",tif_rearrange_param["import_num_frames"])


In [ ]:
#Load and project all data
warnings.filterwarnings('ignore')
import_image_array=np.zeros((tif_rearrange_param['import_height'],tif_rearrange_param['import_width'],len(tif_file_list)),dtype='float32')
times=list()
for f,file in enumerate(tif_file_list):
    fix_time=timeit.default_timer()            
    try:
        del input_image_array, fixed_image_array, fixed_image_array_crop
    except:
        pass
    print("Loading #"+str(f+1)+"/"+str(len(tif_file_list))+": ",tif_file_list[f],end="...")
    input_image_array=load_tiff_stack(tif_file_list[f])
    input_image_array=input_image_array.astype('float32')
    input_image_array=np.moveaxis(input_image_array,[0, 1, 2], [-1, -3, -2])
    print("Projecting",end="...")
    _,_,final_num_frames=np.shape(input_image_array)
    import_image_array[:,:,f] = np.mean(input_image_array[:,:,range(final_num_frames)],axis=2)
    try:
        del input_image_array, fixed_image_array, fixed_image_array_crop
    except:
        pass
    times.append(timeit.default_timer()-fix_time)
    itertime=np.mean(times)
    iterremain=len(tif_file_list)-f
    ETA=str(np.round((iterremain*itertime/60)*10)/10)+"min"
    ETA1=str(np.round(((f)/len(tif_file_list)*100)*10)/10)+"% ETA "+str(np.round((iterremain*itertime/60)*10)/10)+"min"
    avi_time=timeit.default_timer()   
    print("Finished ETA = "+ETA)
write_tif(os.path.join(saveDir,fileID+'_import.tif'), import_image_array)    
#####################################################################
plt.figure(figsize=(40,10))
plt.imshow(np.mean(import_image_array,axis=2))

In [ ]:
DisplayContrast=0.5
projInterval=5000
input_image_array_full_mean=np.mean(input_image_array[:,:,range(tif_rearrange_param["import_num_frames"])],axis=2)
plt.figure(1,figsize=(40,20))
plt.imshow(input_image_array_full_mean,vmax=np.nanmax(input_image_array_full_mean)*DisplayContrast,vmin=0)
plt.plot([tif_rearrange_param['import_width']/2,tif_rearrange_param['import_width']/2],[tif_rearrange_param['import_height'],0],color='red',linewidth=0.5)
plt.title(os.path.basename(tif_file_list[testfilecount])+" Proj Frames "+str(0)+"-"+str(tif_rearrange_param["import_num_frames"]),fontsize=20)
for p in range(0,int(np.floor(tif_rearrange_param["import_num_frames"]/projInterval))):
    tempMean=np.mean(input_image_array[:,:,range(p*projInterval,(p+1)*projInterval)],axis=2)
    plt.figure(figsize=(40,10));
    plt.imshow(tempMean,vmax=np.nanmax(tempMean)*DisplayContrast,vmin=0);
    plt.plot([tif_rearrange_param['import_width']/2,tif_rearrange_param['import_width']/2],[tif_rearrange_param['import_height'],0],color='red',linewidth=0.5);
    plt.title(os.path.basename(tif_file_list[testfilecount])+" Proj Frames "+str(p*projInterval)+"-"+str((p+1)*projInterval),fontsize=20);

In [ ]:
DisplayContrast=0.2
tif_rearrange_param["frameRange"]=[5000,15000]
tif_rearrange_param["frames"] = list(range(tif_rearrange_param["frameRange"][0],tif_rearrange_param["frameRange"][1]))
tif_rearrange_param["nframes"] = len(tif_rearrange_param["frames"])
input_image_array_mean=np.mean(input_image_array[:,:,range(tif_rearrange_param["frameRange"][0],tif_rearrange_param["frameRange"][1])],axis=2)
plt.figure(1,figsize=(40,20));
plt.imshow(input_image_array_mean,vmax=np.nanmax(input_image_array_mean)*DisplayContrast,vmin=0);
plt.title(os.path.basename(tif_file_list[testfilecount])+" "+str(tif_rearrange_param["nframes"])+" Frames",fontsize=20);
plt.plot([tif_rearrange_param['import_width']*0.125,tif_rearrange_param['import_width']*0.125],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=0.5);
plt.plot([tif_rearrange_param['import_width']*0.25,tif_rearrange_param['import_width']*0.25],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=1);
plt.plot([tif_rearrange_param['import_width']*0.333,tif_rearrange_param['import_width']*0.333],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=0.5);
plt.plot([tif_rearrange_param['import_width']*0.375,tif_rearrange_param['import_width']*0.375],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=0.5);
plt.plot([tif_rearrange_param['import_width']*0.5,tif_rearrange_param['import_width']*0.5],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=2);
plt.plot([tif_rearrange_param['import_width']*0.625,tif_rearrange_param['import_width']*0.625],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=0.5);
plt.plot([tif_rearrange_param['import_width']*0.666,tif_rearrange_param['import_width']*0.666],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=0.5);
plt.plot([tif_rearrange_param['import_width']*0.75,tif_rearrange_param['import_width']*0.75],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=1);
plt.plot([tif_rearrange_param['import_width']*0.875,tif_rearrange_param['import_width']*0.875],[tif_rearrange_param['import_height']-1,0],color='red',linewidth=0.5);


In [ ]:
peak1=55
peak2=298
testShift=0
testShifts=[4,4,4, 3, 2]
for line in range(input_image_array_mean.shape[0]):
    plt.figure(figsize=(40,3));
    testLine=input_image_array_mean[line,:]
    testLine_invert=testLine[::-1]
    testLine_midpoint=np.round(testLine.shape[0]/2)
    plt.plot(np.arange(testLine.shape[0]),testLine,label='orig',color='k');
    # plt.plot(np.arange(testLine.shape[0]),testLine_invert,label='invert',color='b');
    plt.plot(testShift+np.arange(testLine.shape[0]),testLine_invert,label='invert +'+str(testShift),color='r');
    plt.plot([testLine_midpoint,testLine_midpoint],[0,np.nanmax(testLine)]);
    plt.plot([peak1,peak1],[0,np.nanmax(testLine)]);
    plt.plot([peak2,peak2],[0,np.nanmax(testLine)]);
    plt.legend();
    plt.title(os.path.basename(tif_file_list[testfilecount])+" Y = "+str(line),fontsize=20);
    time.sleep(0.1)


In [ ]:
testfilecount = 5
print("Loading: ",tif_file_list[testfilecount],end='...')
try:
    input_image_array = load_tiff_stack(tif_file_list[testfilecount])
except:    
    print("ScanImage load not working ",end='...')
    input_image_array=imread(tif_file_list[testfilecount])
print("Finished!")
input_image_array=np.moveaxis(input_image_array,[0, 1, 2], [-1, -3, -2])
tif_rearrange_param["import_height"],tif_rearrange_param["import_width"],tif_rearrange_param["import_num_frames"]=input_image_array.shape
print("IMPORT Image Height = ",tif_rearrange_param["import_height"])
print("IMPORT Image Width = ",tif_rearrange_param["import_width"])
print("IMPORT Num Frames = ",tif_rearrange_param["import_num_frames"])


In [ ]:
#All Bidir correction settings
#Options for rearrangement
tif_rearrange_param['upscale'] = 10
tif_rearrange_param["noise_floor"] = 10 #sets any values below this to zero
tif_rearrange_param["compress_raw"]=0 #0 off or num frames to sum in saving data
tif_rearrange_param["flip_all_rows"]=False # Reverses rows for each cut set
tif_rearrange_param["flip_alt_rows"]=0 #flip direction of each row 0 off 1 = flip rows 1,3,5 <2> flip rows 0,2,4
tif_rearrange_param["line_shift"]=0 #off how many pixels to shift 0 off positive shift forward negative shift back
tif_rearrange_param["line_shift_alt"]=0 #NOT USING NOW #0 off 1 or 2 to pick which alternating row
tif_rearrange_param["truncate_column"]=False #removes single pix prior to split and interleave to get even num columns
tif_rearrange_param["simple_interleave"]=False #cut in half and interleave
tif_rearrange_param["flip_simple_interleave"] = True #flip the second half of the row while interleaving

tif_rearrange_param["initial_resize"] = False
tif_rearrange_param["initial_resize_y"] = 1
tif_rearrange_param["initial_resize_x"] = 1
tif_rearrange_param["initial_resize_method"] = cv2.INTER_LINEAR

tif_rearrange_param["intermediate_resize"] = True
tif_rearrange_param["intermediate_resize_y"] = 1
tif_rearrange_param["intermediate_resize_x"] = tif_rearrange_param['upscale']
tif_rearrange_param["intermediate_resize_method"] = cv2.INTER_LINEAR

tif_rearrange_param["final_resize"] = True
tif_rearrange_param["final_resize_y"] = np.round(tif_rearrange_param["initial_resize_x"]*tif_rearrange_param["intermediate_resize_x"]*ImagingInfo['PixelRatio'])
tif_rearrange_param["final_resize_x"] = 1
tif_rearrange_param["final_resize_method"] = cv2.INTER_CUBIC

#first is the first half of the slice then second is the second half lof the slice to be interleaved
#you can set custom offsets and inversions for both halves
tif_rearrange_param["offset_split_interleave"]=1 #set ranges and inversions below 0 off
                          #1 = 1 cut 2 = 2 cut 3 = 3 cut etc only up to 3 cuts possible right now
tif_rearrange_param["offset_split_collapse"]=False #calculates an nan average for the offset lines too consolidate an image
tif_rearrange_param["offset_split_coord"]=177 #calculates an nan average for the offset lines too consolidate an image
tif_rearrange_param['offset_split_interleave_param']={} 
tif_rearrange_param['offset_split_interleave_param']["First Source"]=range(0,tif_rearrange_param["offset_split_coord"]*tif_rearrange_param["initial_resize_x"])
tif_rearrange_param['offset_split_interleave_param']["First Invert"]=False
tif_rearrange_param['offset_split_interleave_param']["First LeftEdge"]=0
tif_rearrange_param['offset_split_interleave_param']["First Placement"]=\
range(tif_rearrange_param['offset_split_interleave_param']["First LeftEdge"],
      tif_rearrange_param['offset_split_interleave_param']["First LeftEdge"]+
      len(list(tif_rearrange_param['offset_split_interleave_param']["First Source"])))
tif_rearrange_param['offset_split_interleave_param']["Second Source"]=range(tif_rearrange_param["offset_split_coord"]*tif_rearrange_param["initial_resize_x"],tif_rearrange_param["import_width"]*tif_rearrange_param["initial_resize_x"])
tif_rearrange_param['offset_split_interleave_param']["Second Invert"]=True
tif_rearrange_param['offset_split_interleave_param']["Second LeftEdge"]=0
tif_rearrange_param['offset_split_interleave_param']["Second Placement"]=\
range(tif_rearrange_param['offset_split_interleave_param']["Second LeftEdge"],
      tif_rearrange_param['offset_split_interleave_param']["Second LeftEdge"]+
      len(list(tif_rearrange_param['offset_split_interleave_param']["Second Source"])))

tif_rearrange_param["line_pair_correlation_auto_shifting"] = True
tif_rearrange_param["line_pair_correlation_auto_shifting_max_shift"] = 100
tif_rearrange_param["line_pair_correlation_auto_shifting_filter"] = 21

tif_rearrange_param["inter_line_pair_correlation_auto_shifting"] = True
tif_rearrange_param["inter_line_pair_correlation_auto_shifting_direction"] = 1 # 0 top down 1 bottom up
tif_rearrange_param["inter_line_pair_correlation_auto_shifting_max_shift"] = 100
tif_rearrange_param["inter_line_pair_correlation_auto_shifting_filter"] = 21

tif_rearrange_param["all_line_correlation_auto_shifting"] = False
tif_rearrange_param["all_line_correlation_auto_shifting_direction"] = 1 # 0 top down 1 bottom up
tif_rearrange_param["all_line_correlation_auto_shifting_max_shift"] = 20
tif_rearrange_param["all_line_correlation_auto_shifting_filter"] = 21

tif_rearrange_param["manual_bidir_linepair_amp_adjust"] = False
tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"]=[1,0.95]
tif_rearrange_param["manual_bidir_linepair_amp_adjust_scalars"]=np.array([[1, 1], [0.9, 1], [0.9, 1], [1, 0.9], [1, 0.9], [1, 1], [1, 1], [1, 0.8], [1, 1], [1, 1], [1, 0.8], [1, 0.8], [1, 0.8], [1, 0.8], [1, 0.95], [1, 0.95], [1, 1], [1, 1], [1, 1], [1, 1]])

tif_rearrange_param["manual_bidir_linepair_shift_and_scale"] = True
tif_rearrange_param["manual_bidir_linepair_shift"]=0
tif_rearrange_param["manual_bidir_linepair_resize_factor"]=1.033
tif_rearrange_param["manual_bidir_linepair_resize_method"]= cv2.INTER_LINEAR
tif_rearrange_param["manual_bidir_linepair_pairshifts"] = [0,3,0,0,0,0,0,0,0,2,5,2,2,0,-2,0,-1,0,0,0]

tif_rearrange_param["manual_bidir_linepair_shift_and_scale"] = False
tif_rearrange_param["manual_bidir_linepair_shift"]=        [0,0,3,2,   0,   0,   0,   0, -10,-4,-4,-4,-1,  -4,    0,    0,   0,0,0,0]
tif_rearrange_param["manual_bidir_linepair_resize_factor"]=[1,1,1,1,1.04,1.04,1.05,1.04, 1.1, 1, 1, 1, 1,1.03, 1.01, 1.01,1.01,1,1,1]
tif_rearrange_param["manual_bidir_linepair_resize_method"]= cv2.INTER_LINEAR
tif_rearrange_param["manual_bidir_linepair_pairshifts"] = [0,3,0,0,0,0,0,0,0,2,5,2,2,0,-2,0,-1,0,0,0]

tif_rearrange_param["manual_bidir_linepair_shift_and_scale"] = True
tif_rearrange_param["manual_bidir_linepair_shift"]=0
tif_rearrange_param["manual_bidir_linepair_resize_factor"]=1.05
tif_rearrange_param["manual_bidir_linepair_resize_method"]= cv2.INTER_LINEAR
tif_rearrange_param["manual_bidir_linepair_pairshifts"] = 0



In [ ]:
#Check for good settings
if tif_rearrange_param["compress_raw"]>tif_rearrange_param["import_num_frames"]:
    print("turning off compression...")
    tif_rearrange_param["compress_raw"]=0
if tif_rearrange_param["simple_interleave"] and tif_rearrange_param["offset_split_interleave"]:
    warnings.warn('you cant have both interleave and offeset interleave active!')
if tif_rearrange_param["offset_split_interleave"]>0:
    print("Total num pixels per row = ",tif_rearrange_param["import_width"])
    framecount=0;
    print("Checking Custom offset split interleave settings")
    print(("First Source Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["First Source"]))))
    print(("First Placement Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["First Placement"]))))
    framecount=framecount+len(tif_rearrange_param['offset_split_interleave_param']["First Source"])
    if len(tif_rearrange_param['offset_split_interleave_param']["First Source"]) != len(tif_rearrange_param['offset_split_interleave_param']["First Placement"]):
        warnings.warn('First range problem')
    print(("Second Source Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["Second Source"]))))
    print(("Second Placement Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["Second Placement"]))))
    framecount=framecount+len(tif_rearrange_param['offset_split_interleave_param']["Second Source"])
    if len(tif_rearrange_param['offset_split_interleave_param']["Second Source"]) != len(tif_rearrange_param['offset_split_interleave_param']["Second Placement"]):
        warnings.warn('Second range problem')
    if tif_rearrange_param["offset_split_interleave"]>=2:
        print(("Third Source Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["Third Source"]))))
        print(("Third Placement Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["Third Placement"]))))
        framecount=framecount+len(tif_rearrange_param['offset_split_interleave_param']["Third Source"])
        if len(tif_rearrange_param['offset_split_interleave_param']["Third Source"]) != len(tif_rearrange_param['offset_split_interleave_param']["Third Placement"]):
            warnings.warn('Third range problem')
    if tif_rearrange_param["offset_split_interleave"]>=3:
        print(("Fourth Source Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["Fourth Source"]))))
        print(("Fourth Placement Length = "+str(len(tif_rearrange_param['offset_split_interleave_param']["Fourth Placement"]))))
        framecount=framecount+len(tif_rearrange_param['offset_split_interleave_param']["Fourth Source"])
        if len(tif_rearrange_param['offset_split_interleave_param']["Fourth Source"]) != len(tif_rearrange_param['offset_split_interleave_param']["Fourth Placement"]):
            warnings.warn('Fourth range problem')
    print("Total num pixels allocated pixels per row = ",framecount)
    if framecount>tif_rearrange_param["import_width"]:
        warnings.warn('Sourcing too many pixels, check for overlap')

if tif_rearrange_param["offset_split_interleave"]>0:
    print("First Source:")
    print(*tif_rearrange_param['offset_split_interleave_param']["First Source"])
    print("Second Source:")
    print(*tif_rearrange_param['offset_split_interleave_param']["Second Source"])
    if tif_rearrange_param["offset_split_interleave"]>=2:
        print("Third Source:")
        print(*tif_rearrange_param['offset_split_interleave_param']["Third Source"])
    if tif_rearrange_param["offset_split_interleave"]>=3:
        print("Fourth Source:")
        print(*tif_rearrange_param['offset_split_interleave_param']["Fourth Source"])

In [ ]:
#Test single set of parameters
tif_rearrange_param["frameRange"]=[5000,15000]
tif_rearrange_param["frames"] = list(range(tif_rearrange_param["frameRange"][0],tif_rearrange_param["frameRange"][1]))
tif_rearrange_param["nframes"] = len(tif_rearrange_param["frames"])
print(("frameRange = "+str(tif_rearrange_param['frameRange'])))
print(("nframes =    "+str(tif_rearrange_param['nframes'])))
input_image_array_mean=np.mean(input_image_array[:,:,range(tif_rearrange_param["frameRange"][0],tif_rearrange_param["frameRange"][1])],axis=2)
plt.figure(figsize=(50,5))
plt.imshow(input_image_array_mean)
plt.title("Input Mean Proj Frames "+str(tif_rearrange_param["frameRange"][0])+"-"+str(tif_rearrange_param["frameRange"][1]),fontsize=20)
fixed_image_array, fixed_overall_mean_image, fixed_image_height, fixed_image_width= \
single_tif_rearrange(fileID,testfilecount,input_image_array,tif_file_list,tif_rearrange_param)
_,_,final_num_frames=np.shape(fixed_image_array)
print(fixed_image_array.shape)
#############################################################################
contrast=0.4
plt.figure(figsize=(10,10))
plt.imshow(fixed_overall_mean_image,vmin=0,vmax=np.nanmax(fixed_overall_mean_image)*contrast,interpolation='none')

In [ ]:
#Test display
DisplayContrast=0.5
aspectratio=1
%matplotlib inline
fig, (ax1) = plt.subplots(1, 1,figsize=(20,20))
ax1.imshow(fixed_overall_mean_image,vmax=np.nanmax(fixed_overall_mean_image)*DisplayContrast,interpolation='none',vmin=0,aspect=aspectratio)
ax1.title.set_text((os.path.basename(tif_file_list[testfilecount])+" | line_shift = "+str(tif_rearrange_param["line_shift"])+" | offset_split_interleave = "+str(tif_rearrange_param["offset_split_interleave"])))
if (tif_rearrange_param["offset_split_interleave"] or tif_rearrange_param["simple_interleave"]) and not tif_rearrange_param["offset_split_collapse"]:
    if tif_rearrange_param["offset_split_interleave"]:
        num_split_rows=int(tif_rearrange_param["offset_split_interleave"]+1)*(tif_rearrange_param["initial_resize_y"]*tif_rearrange_param["initial_resize_y"]*tif_rearrange_param["final_resize_y"])
    else:
        num_split_rows=2
    temp_height,temp_width=fixed_overall_mean_image.shape
    tempcount=1
    for y in range(temp_height):
        tempcount+=1
        if tempcount==num_split_rows:
            plt.axline([0,y-0.5],[temp_width-1,y-0.5],color='red',lw=0.5)
            tempcount=0
plt.pause(0.1)
#Just display origin and corrected
DisplayContrast=0.8
%matplotlib inline
fig, (ax1,ax2) = plt.subplots(2, 1,figsize=(30,10))
ax1.imshow(input_image_array_mean,interpolation='none',vmax=np.nanmax(input_image_array_mean)*DisplayContrast,vmin=0)
ax1.title.set_text((tif_file_list[testfilecount]+" ORIGINAL IMPORT"))
ax2.imshow(fixed_overall_mean_image,interpolation='none',vmax=np.nanmax(fixed_overall_mean_image)*DisplayContrast,vmin=0)
ax1.title.set_text((tif_file_list[testfilecount]+" | line_shift = "+str(tif_rearrange_param["line_shift"])+" | offset_split_interleave = "+str(tif_rearrange_param["offset_split_interleave"])))
if (tif_rearrange_param["offset_split_interleave"] or tif_rearrange_param["simple_interleave"]) and not tif_rearrange_param["offset_split_collapse"]:
    if tif_rearrange_param["offset_split_interleave"]:
        num_split_rows=int(tif_rearrange_param["offset_split_interleave"]+1)*(tif_rearrange_param["initial_resize_y"]*tif_rearrange_param["initial_resize_y"]*tif_rearrange_param["final_resize_y"])
    else:
        num_split_rows=2
    temp_height,temp_width=fixed_overall_mean_image.shape
    tempcount=1
    for y in range(temp_height):
        tempcount+=1
        if tempcount==num_split_rows:
            plt.axline([0,y-0.5],[temp_width-1,y-0.5],color='red',lw=0.5)
            tempcount=0
plt.pause(0.1)


In [ ]:
contrast=0.4
plt.figure(1,figsize=(20,20))
plt.imshow(fixed_overall_mean_image,vmin=0,vmax=np.nanmax(fixed_overall_mean_image_resize)*contrast,interpolation='none');
print(fixed_overall_mean_image.shape)

In [ ]:
#set crop window to remove assymetric scan artifacts
tif_rearrange_param["crop_data"]=True
crop_param={};
crop_param["fixed_crop_xlim1"]=7
crop_param["fixed_crop_xlim2"]=170
crop_param["fixed_crop_ylim1"]=11
crop_param["fixed_crop_ylim2"]=40
crop_param["fixed_crop_xlim1"]=50
crop_param["fixed_crop_xlim2"]=1600
crop_param["fixed_crop_ylim1"]=400
crop_param["fixed_crop_ylim2"]=1760
if tif_rearrange_param["crop_data"]:
    print("X Limits: ",crop_param["fixed_crop_xlim1"]," - ",crop_param["fixed_crop_xlim2"])
    print("Y Limits: ",crop_param["fixed_crop_ylim1"]," - ",crop_param["fixed_crop_ylim2"])
tif_rearrange_param["crop_param"]=crop_param
################
#Test Crop
if tif_rearrange_param["crop_data"]:
    fixed_overall_mean_image_crop=fixed_overall_mean_image[crop_param["fixed_crop_ylim1"]:crop_param["fixed_crop_ylim2"],
                                             crop_param["fixed_crop_xlim1"]:crop_param["fixed_crop_xlim2"]]
else:
    fixed_overall_mean_image_crop=fixed_overall_mean_image
tif_rearrange_param['final_height']=fixed_overall_mean_image_crop.shape[0]
tif_rearrange_param['final_width']=fixed_overall_mean_image_crop.shape[1]
plt.figure(1,figsize=(30,20))
plt.title(tif_file_list[testfilecount])
plt.imshow(fixed_overall_mean_image_crop,interpolation='none',vmin=0,vmax=np.nanmax(fixed_overall_mean_image_crop)*contrast)
print(np.shape(fixed_overall_mean_image_crop))
tif_rearrange_param["final_image_crop_height"],tif_rearrange_param["final_image_crop_width"]=fixed_overall_mean_image_crop.shape
tif_rearrange_param["final_image_num_frames"]=fixed_image_array.shape[2]

In [ ]:
#Project the data for just part of the dataset to avoid drift
tif_rearrange_param["frameRange"]=[5000,15000]
tif_rearrange_param["frames"] = list(range(tif_rearrange_param["frameRange"][0],tif_rearrange_param["frameRange"][1]))
tif_rearrange_param["nframes"] = len(tif_rearrange_param["frames"])
warnings.filterwarnings('ignore')
print(("frameRange = "+str(tif_rearrange_param['frameRange'])))
print(("nframes =    "+str(tif_rearrange_param['nframes'])))
final_input_image_array=np.zeros((tif_rearrange_param['import_height'],tif_rearrange_param['import_width'],len(tif_file_list)),dtype='float32')
final_output_image_array=np.zeros((tif_rearrange_param['final_height'],tif_rearrange_param['final_width'],len(tif_file_list)),dtype='float32')
print(final_input_image_array.shape)
print(final_output_image_array.shape)
times=list()
for f in range(ImagingInfo['nSlices']):
    file=tif_file_list[f]
    fix_time=timeit.default_timer()            
    try:
        del input_image_array, fixed_image_array, fixed_image_array_crop
    except:
        pass
    print("Loading #"+str(f+1)+"/"+str(len(tif_file_list))+": ",file,end="...")
    input_image_array=load_tiff_stack(file)
    input_image_array=input_image_array.astype('float32')
    input_image_array=np.moveaxis(input_image_array,[0, 1, 2], [-1, -3, -2])
    input_image_array_mean=np.nanmean(input_image_array,axis=2)
    final_input_image_array[:,:,f] = copy.deepcopy(np.nanmean(input_image_array[:,:,tif_rearrange_param["frames"]],axis=2))
    print("input_image_array.shape = "+str(input_image_array.shape))
    print("Rearranging...")
    
    fixed_image_array, fixed_overall_mean_image, fixed_image_height, fixed_image_width= \
    single_tif_rearrange(fileID,f,input_image_array,tif_file_list,tif_rearrange_param,False)
    _,_,final_num_frames=np.shape(fixed_image_array)
    
    print("Cropping and Projecting",end="...")
    
    if tif_rearrange_param["crop_data"]:
        fixed_image_array_crop=copy.deepcopy(fixed_image_array[crop_param["fixed_crop_ylim1"]:crop_param["fixed_crop_ylim2"],
                                                 crop_param["fixed_crop_xlim1"]:crop_param["fixed_crop_xlim2"],:])
        fixed_overall_mean_image=copy.deepcopy(fixed_overall_mean_image[crop_param["fixed_crop_ylim1"]:crop_param["fixed_crop_ylim2"],
                                                 crop_param["fixed_crop_xlim1"]:crop_param["fixed_crop_xlim2"]])
    else:
        fixed_image_array_crop=copy.deepcopy(fixed_image_array)
        fixed_overall_mean_image=copy.deepcopy(fixed_overall_mean_image)
    final_output_image_array[:,:,f] = copy.deepcopy(fixed_overall_mean_image)
    print("fixed_image_array.shape = "+str(fixed_image_array.shape),end='...')
    print("fixed_image_array_crop.shape = "+str(fixed_image_array_crop.shape),end='...')
    DisplayContrast=0.8
    plt.ion();
    fig, (ax1,ax2) = plt.subplots(2, 1,figsize=(30,10));
    ax1.imshow(input_image_array_mean,interpolation='none',vmax=np.nanmax(input_image_array_mean)*DisplayContrast,vmin=0);
    ax1.title.set_text((file+" ORIGINAL IMPORT "+str(tif_rearrange_param["nframes"])+" Frames"));
    ax2.imshow(fixed_overall_mean_image,interpolation='none',vmax=np.nanmax(fixed_overall_mean_image)*DisplayContrast,vmin=0);
    ax1.title.set_text((file+" | line_shift = "+str(tif_rearrange_param["line_shift"])+" | offset_split_interleave = "+str(tif_rearrange_param["offset_split_interleave"])));
    if (tif_rearrange_param["offset_split_interleave"] or tif_rearrange_param["simple_interleave"]) and not tif_rearrange_param["offset_split_collapse"]:
        if tif_rearrange_param["offset_split_interleave"]:
            num_split_rows=int(tif_rearrange_param["offset_split_interleave"]+1)*(tif_rearrange_param["initial_resize_y"]*tif_rearrange_param["initial_resize_y"]*tif_rearrange_param["final_resize_y"])
        else:
            num_split_rows=2
        temp_height,temp_width=fixed_overall_mean_image.shape
        tempcount=1
        for y in range(temp_height):
            tempcount+=1
            if tempcount==num_split_rows:
                plt.axline([0,y-0.5],[temp_width-1,y-0.5],color='red',lw=0.5);
                tempcount=0    
    plt.show();
    plt.gcf().canvas.draw();
    time.sleep(0.1);
    try:
        del input_image_array, fixed_image_array, fixed_image_array_crop, fixed_overall_mean_image
    except:
        pass
    times.append(timeit.default_timer()-fix_time)
    itertime=np.mean(times)
    iterremain=len(tif_file_list)-f
    ETA=str(np.round((iterremain*itertime/60)*10)/10)+"min"
    ETA1=str(np.round(((f+1)/len(tif_file_list)*100)*10)/10)+"% ETA "+str(np.round((iterremain*itertime/60)*10)/10)+"min"
write_tif(os.path.join(saveDir,fileID+'_frs_'+str(tif_rearrange_param["frameRange"][0])+'-'+str(tif_rearrange_param["frameRange"][1])+'_fixed_input.tif'), final_input_image_array)    
write_tif(os.path.join(saveDir,fileID+'_frs_'+str(tif_rearrange_param["frameRange"][0])+'-'+str(tif_rearrange_param["frameRange"][1])+'_fixed_output.tif'), final_output_image_array)    
#####################################################################

In [ ]:
tif_rearrange_param['final_resize_method'] = cv2.INTER_CUBIC
tif_rearrange_param['final_height_resize'] = int(np.round(tif_rearrange_param['final_height']/tif_rearrange_param['upscale']))
tif_rearrange_param['final_width_resize'] = int(np.round(tif_rearrange_param['final_width']/tif_rearrange_param['upscale']))
tif_rearrange_param['final_depth_resize'] = int(np.round(ImagingInfo['nSlices']*ImagingInfo['VoxelRatio']))
print("final_height_resize = "+str(tif_rearrange_param['final_height_resize']))
print("final_width_resize =  "+str(tif_rearrange_param['final_width_resize']))
print("final_depth_resize =  "+str(tif_rearrange_param['final_depth_resize']))

In [ ]:
final_output_image_array_resize_temp = np.zeros((tif_rearrange_param['final_height_resize'], tif_rearrange_param['final_width_resize'], ImagingInfo['nSlices']),dtype='float32')
final_output_image_array_resize = np.zeros((tif_rearrange_param['final_height_resize'], tif_rearrange_param['final_width_resize'], tif_rearrange_param['final_depth_resize']),dtype='float32')
for z in range(ImagingInfo['nSlices']):
    final_output_image_array_resize_temp[:,:,z] = cv2.resize(final_output_image_array[:,:,z], dsize=(tif_rearrange_param['final_width_resize'],tif_rearrange_param['final_height_resize']), interpolation=tif_rearrange_param['final_resize_method'])
write_tif(os.path.join(saveDir,fileID+'_frs_'+str(tif_rearrange_param["frameRange"][0])+'-'+str(tif_rearrange_param["frameRange"][1])+'_fixed_output_xy_resize.tif'), final_output_image_array_resize_temp)    
for y in range(tif_rearrange_param['final_height_resize']):
    final_output_image_array_resize[y,:,:] = cv2.resize(final_output_image_array_resize_temp[y,:,:], dsize=(tif_rearrange_param['final_depth_resize'],tif_rearrange_param['final_width_resize']), interpolation=tif_rearrange_param['final_resize_method'])
write_tif(os.path.join(saveDir,fileID+'_frs_'+str(tif_rearrange_param["frameRange"][0])+'-'+str(tif_rearrange_param["frameRange"][1])+'_fixed_output_xyz_resize.tif'), final_output_image_array_resize)    

In [ ]:
#Collect profile for one bead
profile_coord = [43, 100, 125] # y x z
profile_max=61
maxVal = 60

xy_image = final_output_image_array_resize[:,:,profile_coord[2]]
xz_image = np.transpose(final_output_image_array_resize[:,profile_coord[1],:])
yz_image = np.transpose(final_output_image_array_resize[profile_coord[0],:,:])
fig, ax = plt.subplots(1, 3,figsize=(40,10));
fig.tight_layout()
ax[0].imshow(xy_image,vmin=0,vmax=np.nanmax(xy_image),cmap='gray',interpolation='none')
ax[0].plot(profile_coord[1],profile_coord[0],'x',markersize=30,color='red')
ax[1].imshow(xz_image,vmin=0,vmax=np.nanmax(xz_image),cmap='gray',interpolation='none')
ax[1].plot(profile_coord[0],profile_coord[2],'x',markersize=30,color='red')
ax[2].imshow(yz_image,vmin=0,vmax=np.nanmax(yz_image),cmap='gray',interpolation='none')
ax[2].plot(profile_coord[1],profile_coord[2],'x',markersize=30,color='red')


In [ ]:
#Fig 7b-e
saveFig=True
x_color = (1,0,0)
y_color = (0,0.75,0.75)
z_color = (0,0.75,0)
cmap='gray'
scalebar_um = 2
scalebar_coord_offset = 10
xlim=[-3,3]
xlim=[-2,2]
xlim=[-1.5,1.5]

imXY_xlim=[0,155]
imXY_ylim=[136,1]

x_profile = final_output_image_array_resize[profile_coord[0],:,profile_coord[2]]
x_profile_max = np.nanmax(x_profile[profile_coord[1]+int(np.floor(xlim[0]/ImagingInfo['PixelWidth'])):profile_coord[1]+int(np.floor(xlim[1]/ImagingInfo['PixelWidth']))])
x_profile_min = np.nanmin(x_profile[profile_coord[1]+int(np.floor(xlim[0]/ImagingInfo['PixelWidth'])):profile_coord[1]+int(np.floor(xlim[1]/ImagingInfo['PixelWidth']))])
x_profile_halfmax = (x_profile_max-x_profile_min)/2+x_profile_min
x_profile_x = (np.arange(len(x_profile))-np.argmax(x_profile))*ImagingInfo['PixelWidth']
x_profile_halfmax_low = (np.abs(x_profile[0:np.argmax(x_profile)] - x_profile_halfmax)).argsort()
x_profile_halfmax_low_y = x_profile[(np.abs(x_profile[0:np.argmax(x_profile)] - x_profile_halfmax)).argmin()]
x_profile_halfmax_low_y = np.mean(x_profile[x_profile_halfmax_low[0:2]])
x_profile_halfmax_low_x = x_profile_x[(np.abs(x_profile[0:np.argmax(x_profile)] - x_profile_halfmax)).argmin()]
x_profile_halfmax_high = (np.abs(x_profile[np.argmax(x_profile):-1] - x_profile_halfmax)).argsort()
x_profile_halfmax_high_y = x_profile[(np.abs(x_profile[np.argmax(x_profile):-1] - x_profile_halfmax)).argmin()+np.argmax(x_profile)]
x_profile_halfmax_high_y = np.mean(x_profile[x_profile_halfmax_high[0:2]+np.argmax(x_profile)])
x_profile_halfmax_high_x = x_profile_x[(np.abs(x_profile[np.argmax(x_profile):-1] - x_profile_halfmax)).argmin()+np.argmax(x_profile)]
x_profile_halfmax_y = np.nanmean([x_profile_halfmax_high_y,x_profile_halfmax_low_y])
x_profile_FWHM = x_profile_halfmax_high_x-x_profile_halfmax_low_x
print(x_profile_halfmax)
print(x_profile_FWHM)
fig, ax = plt.subplots(1, 3,figsize=(15,5));
ax[0].plot(x_profile_x,x_profile)
ax[0].plot([x_profile_halfmax_low_x,x_profile_halfmax_high_x],[x_profile_halfmax_y,x_profile_halfmax_y])


y_profile = final_output_image_array_resize[:,profile_coord[1],profile_coord[2]]
y_profile_max = np.nanmax(y_profile[profile_coord[0]+int(np.floor(xlim[0]/ImagingInfo['PixelWidth'])):profile_coord[0]+int(np.floor(xlim[1]/ImagingInfo['PixelWidth']))])
y_profile_min = np.nanmin(y_profile[profile_coord[0]+int(np.floor(xlim[0]/ImagingInfo['PixelWidth'])):profile_coord[0]+int(np.floor(xlim[1]/ImagingInfo['PixelWidth']))])
y_profile_halfmax = (y_profile_max-y_profile_min)/2+y_profile_min
y_profile_x = (np.arange(len(y_profile))-np.argmax(y_profile))*ImagingInfo['PixelWidth']
y_profile_halfmax_low = (np.abs(y_profile[0:np.argmax(y_profile)] - y_profile_halfmax)).argsort()
y_profile_halfmax_low_y = y_profile[(np.abs(y_profile[0:np.argmax(y_profile)] - y_profile_halfmax)).argmin()]
y_profile_halfmax_low_y = np.mean(y_profile[y_profile_halfmax_low[0:2]])
y_profile_halfmax_low_x = y_profile_x[(np.abs(y_profile[0:np.argmax(y_profile)] - y_profile_halfmax)).argmin()]
y_profile_halfmax_high = (np.abs(y_profile[np.argmax(y_profile):-1] - y_profile_halfmax)).argsort()
y_profile_halfmax_high_y = y_profile[(np.abs(y_profile[np.argmax(y_profile):-1] - y_profile_halfmax)).argmin()+np.argmax(y_profile)]
y_profile_halfmax_high_y = np.mean(y_profile[y_profile_halfmax_high[0:2]+np.argmax(y_profile)])
y_profile_halfmax_high_x = y_profile_x[(np.abs(y_profile[np.argmax(y_profile):-1] - y_profile_halfmax)).argmin()+np.argmax(y_profile)]
y_profile_halfmax_y = np.nanmean([y_profile_halfmax_high_y,y_profile_halfmax_low_y])
y_profile_FWHM = y_profile_halfmax_high_x-y_profile_halfmax_low_x
print(y_profile_halfmax)
print(y_profile_FWHM)
ax[1].plot(y_profile_x,y_profile)
ax[1].plot([y_profile_halfmax_low_x,y_profile_halfmax_high_x],[y_profile_halfmax_y,y_profile_halfmax_y])

z_profile = final_output_image_array_resize[profile_coord[0],profile_coord[1],:]
z_profile_max = np.nanmax(z_profile)
z_profile_min = np.nanmin(z_profile)
z_profile_halfmax = (z_profile_max-z_profile_min)/2+z_profile_min
z_profile_x = np.arange(len(z_profile))*ImagingInfo['PixelWidth']
z_profile_halfmax_low = (np.abs(z_profile[0:np.argmax(z_profile)] - z_profile_halfmax)).argsort()
z_profile_halfmax_low_y = z_profile[(np.abs(z_profile[0:np.argmax(z_profile)] - z_profile_halfmax)).argmin()]
z_profile_halfmax_low_y = np.mean(z_profile[z_profile_halfmax_low[0:2]])
z_profile_halfmax_low_x = z_profile_x[(np.abs(z_profile[0:np.argmax(z_profile)] - z_profile_halfmax)).argmin()]
z_profile_halfmax_high = (np.abs(z_profile[np.argmax(z_profile):-1] - z_profile_halfmax)).argsort()
z_profile_halfmax_high_y = z_profile[(np.abs(z_profile[np.argmax(z_profile):-1] - z_profile_halfmax)).argmin()+np.argmax(z_profile)]
z_profile_halfmax_high_y = np.mean(z_profile[z_profile_halfmax_high[0:2]+np.argmax(z_profile)])
z_profile_halfmax_high_x = z_profile_x[(np.abs(z_profile[np.argmax(z_profile):-1] - z_profile_halfmax)).argmin()+np.argmax(z_profile)]
z_profile_halfmax_y = np.nanmean([z_profile_halfmax_high_y,z_profile_halfmax_low_y])
z_profile_FWHM = z_profile_halfmax_high_x-z_profile_halfmax_low_x
print(z_profile_halfmax)
print(z_profile_FWHM)
ax[2].plot(z_profile_x,z_profile)
ax[2].plot([z_profile_halfmax_low_x,z_profile_halfmax_high_x],[z_profile_halfmax_y,z_profile_halfmax_y])

fig, ax = plt.subplots(1, 5,figsize=(50,10));
fig.tight_layout()
ax[0].imshow(xy_image,vmin=0,vmax=maxVal,cmap=cmap,interpolation='none')
ax[0].plot([profile_coord[1]+xlim[0]/ImagingInfo['PixelWidth'],profile_coord[1]+xlim[1]/ImagingInfo['PixelWidth']],[profile_coord[0],profile_coord[0]],':',color=x_color,linewidth=2)
ax[0].plot([profile_coord[1],profile_coord[1]],[profile_coord[0]+xlim[0]/ImagingInfo['PixelWidth'],profile_coord[0]+xlim[1]/ImagingInfo['PixelWidth']],':',color=y_color,linewidth=2)
ax[0].plot([scalebar_coord_offset,scalebar_coord_offset+scalebar_um/ImagingInfo['PixelWidth']],np.ones(2)*(xy_image.shape[0]-scalebar_coord_offset),'-',color=(1,1,1),linewidth=3)
ax[0].text(np.mean([scalebar_coord_offset,scalebar_coord_offset+scalebar_um/ImagingInfo['PixelWidth']]),(xy_image.shape[0]-scalebar_coord_offset-2),"2 μm",color=(1,1,1),fontsize=28,ha='center',va='bottom')
ax[0].set_ylim(imXY_ylim)
ax[0].set_xlim(imXY_xlim)

ax[0].set_title('XY',fontsize=28)
imshow_cleanup(ax[0])

ax[1].imshow(xz_image,vmin=0,vmax=maxVal,cmap=cmap,interpolation='none')
ax[1].plot([profile_coord[0],profile_coord[0]],[0,xz_image.shape[0]-1],':',color=z_color,linewidth=2)
ax[1].plot([scalebar_coord_offset,scalebar_coord_offset+scalebar_um/ImagingInfo['PixelWidth']],np.ones(2)*(xz_image.shape[0]-scalebar_coord_offset),'-',color=(1,1,1),linewidth=3)
ax[1].text(np.mean([scalebar_coord_offset,scalebar_coord_offset+scalebar_um/ImagingInfo['PixelWidth']]),(xz_image.shape[0]-scalebar_coord_offset-2),"2 μm",color=(1,1,1),fontsize=28,ha='center',va='bottom')
ax[1].set_title('YZ',fontsize=28)
imshow_cleanup(ax[1])

ax[2].imshow(yz_image,vmin=0,vmax=maxVal,cmap=cmap,interpolation='none')
ax[2].plot([profile_coord[1],profile_coord[1]],[0,yz_image.shape[0]-1],':',color=z_color,linewidth=2)
ax[2].plot([scalebar_coord_offset,scalebar_coord_offset+scalebar_um/ImagingInfo['PixelWidth']],np.ones(2)*(yz_image.shape[0]-scalebar_coord_offset),'-',color=(1,1,1),linewidth=3)
ax[2].text(np.mean([scalebar_coord_offset,scalebar_coord_offset+scalebar_um/ImagingInfo['PixelWidth']]),(yz_image.shape[0]-scalebar_coord_offset-2),"2 μm",color=(1,1,1),fontsize=28,ha='center',va='bottom')
ax[2].set_title('XZ',fontsize=28)
imshow_cleanup(ax[2])

ax[3].plot(x_profile_x,x_profile,label='horz. profile\nFWHM = '+str(np.round(x_profile_FWHM,decimals=2))+' μm',color=x_color,linewidth=1)
ax[3].plot([x_profile_halfmax_low_x,x_profile_halfmax_high_x],[x_profile_halfmax_y,x_profile_halfmax_y],':',color=x_color,linewidth=1)

ax[3].plot(y_profile_x,y_profile,label='vert. profile\nFWHM = '+str(np.round(y_profile_FWHM,decimals=2))+' μm',color=y_color,linewidth=1)
ax[3].plot([y_profile_halfmax_low_x,y_profile_halfmax_high_x],[y_profile_halfmax_y,y_profile_halfmax_y],':',color=y_color,linewidth=1)
ax[3].tick_params(axis='both', which='major', labelsize=24)
ax[3].legend(fontsize=20,loc='upper right')
ax[3].set_xlabel("distance from peak (μm)",fontsize=28)
ax[3].set_ylabel('intensity profile (A.U.)',fontsize=28)
ax[3].set_xlim(xlim)
ax[3].spines['top'].set_visible(False)
ax[3].spines['right'].set_visible(False)
ax[3].get_xaxis().tick_bottom()
ax[3].set_ylim([0,np.around(np.ceil(np.nanmax([np.nanmax(x_profile),np.nanmax(y_profile)]))/5, decimals=1)*5])
ax[3].set_ylim([0,profile_max])
ax[3].set_box_aspect(0.5)

ax[4].plot(z_profile,z_profile_x,label='depth profile\nFWHM = '+str(np.round(z_profile_FWHM,decimals=2))+' μm',color=z_color,linewidth=1)
ax[4].plot([z_profile_halfmax_y,z_profile_halfmax_y],[z_profile_halfmax_low_x,z_profile_halfmax_high_x],':',color=z_color,linewidth=1)
ax[4].tick_params(axis='both', which='major', labelsize=24)
ax[4].legend(fontsize=20,loc='upper right')
ax[4].set_ylabel("z-axis depth (μm)",fontsize=28)
ax[4].set_xlabel('intensity profile (A.U.)',fontsize=28)
ax[4].invert_yaxis()
ax[4].spines['top'].set_visible(False)
ax[4].spines['right'].set_visible(False)
ax[4].get_xaxis().tick_bottom()
ax[4].get_yaxis().tick_left()
ax[4].set_xlim([0,np.around(np.nanmax(z_profile)/5, decimals=1)*5])
ax[4].set_xlim([0,profile_max])
ax[4].set_box_aspect(2)
if saveFig:
    fname=fileID+'_frs_'+str(tif_rearrange_param["frameRange"][0])+'-'+str(tif_rearrange_param["frameRange"][1])+'_fixed_output_'+'y'+str(profile_coord[1])+'-'+'x'+str(profile_coord[0])+'-'+'z'+str(profile_coord[2])+'.pdf'
    print(fname)
    with PdfPages('C:/Users/newmanza/Documents/KerlinLab/Papers/EOD_paper/Fig_iPy_nbs/psfs/'+fname) as pdf:
        pdf.savefig(fig,bbox_inches='tight',pad_inches=0.05,dpi=600)  # saves the current figure into a pdf page
